# BDA Project: Create Your Own Spotify Experience

### Feature Extraction and Transformation


#### Group members:

- Aaqib Ahmed Nazir (i22-1920),
- Arhum Khan (i22-1967),
- Ammar Khasif (i22-1968)

##### Section: DS-D


#### Libraries Used:


In [1]:
import os
import pymongo
import librosa
import numpy as np
import pandas as pd
from tqdm import tqdm
from joblib import Parallel, delayed, Memory
from sklearn.preprocessing import MinMaxScaler

### Loading the file paths

In [2]:
tracks = pd.read_csv('fma_metadata/tracks.csv')
genres = pd.read_csv('fma_metadata/genres.csv')
features = pd.read_csv('fma_metadata/features.csv')
echonest = pd.read_csv('fma_metadata/echonest.csv')
raw_albums = pd.read_csv('fma_metadata/raw_albums.csv')
raw_artists = pd.read_csv('fma_metadata/raw_artists.csv')
raw_genres = pd.read_csv('fma_metadata/raw_genres.csv')
raw_tracks = pd.read_csv('fma_metadata/raw_tracks.csv')
raw_echonest = pd.read_csv('fma_metadata/raw_echonest.csv')

/tmp/ipykernel_4219/3360183426.py:1: DtypeWarning: Columns (0,1,5,6,8,12,18,20,21,22,24,33,34,38,39,44,47,49) have mixed types. Specify dtype option on import or set low_memory=False.
  tracks = pd.read_csv('fma_metadata/tracks.csv')
/tmp/ipykernel_4219/3360183426.py:3: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,

### Function to get artist name 
for testing purposes

In [3]:
def find_artist(artist_name):
    artist_name = artist_name.lower()
    raw_artists["artist_name"] = raw_artists["artist_name"].str.lower()

    # Checking if artist is in the dataset
    if artist_name.lower() in raw_artists["artist_name"].values:
        print("Artist found")
    else:
        print("Artist not found")


artist_name = "lucky dragons"
find_artist(artist_name)

Artist found


### Function to extract features from the audio files and Normalize them
loads an audio file using Librosa library, then extracts three features: MFCC, spectral centroid, and zero-crossing rate. It caches the results for faster access later. If an error occurs, it prints an error message and returns empty arrays.

In [4]:
def extract_features(file):
    try:
        y, sr = librosa.load(file, sr=None)
        mfcc = librosa.feature.mfcc(y=y, sr=sr)
        spectral_centroid = librosa.feature.spectral_centroid(y=y, sr=sr)
        zero_crossing_rate = librosa.feature.zero_crossing_rate(y)
        
        # Normalizing features
        scaler = MinMaxScaler()
        mfcc_normalized = scaler.fit_transform(mfcc.T).T.tolist()
        spectral_centroid_normalized = scaler.fit_transform(spectral_centroid.T).T.tolist()
        zero_crossing_rate_normalized = scaler.fit_transform(zero_crossing_rate.T).T.tolist()
        
        return {"file_name": os.path.basename(file), 
                "mfcc": mfcc_normalized, 
                "spectral_centroid": spectral_centroid_normalized, 
                "zero_crossing_rate": zero_crossing_rate_normalized}
    except Exception as e:
        print(f"Error loading file {file}: {e}")
        return None

### Parallel Audio Feature Extraction and Adding the Features to MongoDB 
utilizes parallel processing to extract audio features from each file concurrently, enhancing computational efficiency. It employs the os.walk function for directory traversal, Parallel from joblib for parallelism, and tqdm for a progress bar display.

In [5]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["audio_features"]
collection = db["features"]

AUDIO_DIR = r"fma_small1"

# Getting all audio files
audio_files = []
for root, dirs, files in os.walk(AUDIO_DIR):
    for file in files:
        if file.endswith(".mp3"):
            audio_files.append(os.path.join(root, file))
            
# Using joblib to extract features in parallel
features = Parallel(n_jobs=-1)(
    delayed(extract_features)(file) for file in tqdm(audio_files, total=len(audio_files))
)
            
features = [f for f in features if f is not None]

# Insert features into MongoDB
collection.insert_many(features)

  0%|          | 0/386 [00:00<?, ?it/s]

100%|██████████| 386/386 [00:42<00:00,  9.02it/s]


InsertManyResult([ObjectId('6638cfe190e9d96e82562aaf'), ObjectId('6638cfe190e9d96e82562ab0'), ObjectId('6638cfe190e9d96e82562ab1'), ObjectId('6638cfe190e9d96e82562ab2'), ObjectId('6638cfe190e9d96e82562ab3'), ObjectId('6638cfe190e9d96e82562ab4'), ObjectId('6638cfe190e9d96e82562ab5'), ObjectId('6638cfe190e9d96e82562ab6'), ObjectId('6638cfe190e9d96e82562ab7'), ObjectId('6638cfe190e9d96e82562ab8'), ObjectId('6638cfe190e9d96e82562ab9'), ObjectId('6638cfe190e9d96e82562aba'), ObjectId('6638cfe190e9d96e82562abb'), ObjectId('6638cfe190e9d96e82562abc'), ObjectId('6638cfe190e9d96e82562abd'), ObjectId('6638cfe190e9d96e82562abe'), ObjectId('6638cfe190e9d96e82562abf'), ObjectId('6638cfe190e9d96e82562ac0'), ObjectId('6638cfe190e9d96e82562ac1'), ObjectId('6638cfe190e9d96e82562ac2'), ObjectId('6638cfe190e9d96e82562ac3'), ObjectId('6638cfe190e9d96e82562ac4'), ObjectId('6638cfe190e9d96e82562ac5'), ObjectId('6638cfe190e9d96e82562ac6'), ObjectId('6638cfe190e9d96e82562ac7'), ObjectId('6638cfe190e9d96e82562a

### Printing the extracted features 
for testing purposes

In [1]:
# priting the first 5 records
# for feature in collection.find().limit(5):
#     print(feature)